### Przetwarzania plików CoNLL-U

W formacie CoNLL-U każdemu segmentowi odpowiada 1 linia, która składa się z kolumn:

    ID: numer kolejny słowa w zdaniu,
    ORTH: forma tekstowa,
    LEMMA: lemat,
    UPOS: uniwersalna część mowy,
    XPOS: tag właściwy dla danego języka,
    UFEATS: uniwersalne cechy morfologiczne,
    HEAD: id nadrzędnika,
    DEPREL: etykieta relacji zależnościowej.

In [1]:
# Na potrzeby tej lekcji utorzymy 2 foldery
import os
os.makedirs(r'nlp_task/source', exist_ok=True)
os.makedirs(r'nlp_task/predictions', exist_ok=True)

In [2]:
import spacy
nlp = spacy.load("pl_core_news_sm")

In [3]:
doc = nlp("Kupiliśmy nowy samochód.")

Podstawowy moduł nie rozdzieli słowa "kupiliśmy"

In [4]:
print("{:20} {:20} {:20}".format('SEGMENT', 'LEMAT', 'UPOS'))
for token in doc:
    print("{:20} {:20} {:20}".format(token.text, token.lemma_, token.pos_))

SEGMENT              LEMAT                UPOS                
Kupiliśmy            kupić                VERB                
nowy                 nowy                 ADJ                 
samochód             samochód             NOUN                
.                    .                    PUNCT               


In [5]:
# ! pip3 install spacy-conll

In [6]:
# Biblioteka spacy_conll służy do przetwarzania wstępnie posegmentowanych plików. 
# Funkcja init_parser posiada argument is_tokenized (True = nie należy przeprowadzać segmentacji)
from spacy_conll import init_parser
nlp = init_parser('spacy', 'pl_core_news_sm', is_tokenized=True)

In [7]:
doc = nlp("Kupili śmy nowy samochód .")

In [8]:
print("{:20} {:20} {:20}".format('SEGMENT', 'LEMAT', 'UPOS'))
for token in doc:
    print("{:20} {:20} {:20}".format(token.text, token.lemma_, token.pos_))

SEGMENT              LEMAT                UPOS                
Kupili               kupić                VERB                
śmy                  być                  AUX                 
nowy                 nowy                 ADJ                 
samochód             samochód             NOUN                
.                    .                    PUNCT               


Przetwarzanie stringa i zapisywanie wyniku do pliku przy pomocy wiersza poleceń

In [9]:
!parse-as-conll --model_or_lang 'pl_core_news_sm' --input_str 'Kupiliśmy nowy samochód.' --output_file 'file_1.conllu' --include_headers

In [10]:
! head file_1.conllu

# sent_id = 1
# text = Kupiliśmy nowy samochód.
1	Kupiliśmy	kupić	VERB	PRAET	_	0	ROOT	_	_
2	nowy	nowy	ADJ	ADJ	_	3	amod	_	_
3	samochód	samochód	NOUN	SUBST	_	1	obj	_	SpaceAfter=No
4	.	.	PUNCT	INTERP	_	1	punct	_	SpaceAfter=No


### Pobierz plik txt i zapisz go do folderu "source"

In [11]:
!wget -P nlp_task/source https://wolnelektury.pl/media/book/txt/korczak-krol-macius-na-wyspie-bezludnej.txt

--2021-01-02 12:39:40--  https://wolnelektury.pl/media/book/txt/korczak-krol-macius-na-wyspie-bezludnej.txt
Resolving wolnelektury.pl (wolnelektury.pl)... 51.83.143.148, 2001:41d0:602:3294::
Connecting to wolnelektury.pl (wolnelektury.pl)|51.83.143.148|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 326911 (319K) [text/plain]
Saving to: ‘nlp_task/source/korczak-krol-macius-na-wyspie-bezludnej.txt’

korczak-krol-macius 100%[===================>] 319,25K  --.-KB/s    in 0,09s   

2021-01-02 12:39:40 (3,55 MB/s) - ‘nlp_task/source/korczak-krol-macius-na-wyspie-bezludnej.txt’ saved [326911/326911]



In [12]:
!head -n 10 nlp_task/source/korczak-krol-macius-na-wyspie-bezludnej.txt

In [13]:
! time parse-as-conll --model_or_lang 'pl_core_news_sm' --input_file 'nlp_task/source/korczak-krol-macius-na-wyspie-bezludnej.txt' --output_file 'nlp_task/predictions/file.conllu' --include_headers
# real = czas w którym przetworzone zostało 5470 zdań.


real	1m1,877s
user	0m57,427s
sys	0m4,907s


In [14]:
# zdania oddzielane są pustą linią, sent_id = numer zdania. text = treść zdania
! tail -n 50 nlp_task/predictions/file.conllu

6	Praw	prawo	NOUN	SUBST	_	5	nmod	_	_
7	Dziecka	dziecko	NOUN	SUBST	_	6	nmod	_	SpaceAfter=No
8	.	.	PUNCT	INTERP	_	2	punct	_	_

# sent_id = 5468
# text = Reprodukcja cyfrowa wykonana przez Fundację Nowoczesna Polska.
1	Reprodukcja	reprodukcja	NOUN	SUBST	_	3	nsubj	_	_
2	cyfrowa	cyfrowy	ADJ	ADJ	_	1	amod	_	_
3	wykonana	wykonać	VERB	PPAS	_	0	ROOT	_	_
4	przez	przez	ADP	PREP	_	5	case	_	_
5	Fundację	fundacja	NOUN	SUBST	_	3	obl:arg	_	_
6	Nowoczesna	nowoczesny	ADJ	ADJ	_	5	nmod	_	_
7	Polska	polska	NOUN	SUBST	_	5	amod	_	SpaceAfter=No
8	.	.	PUNCT	INTERP	_	3	punct	_	SpaceAfter=No

# sent_id = 5469
# text = Opracowanie redakcyjne i przypisy: Paulina Choromańska, Agnieszka Gąsior, Wojciech Kotwica, Dorota Kowalska, Paweł Kozioł.
1	Opracowanie	opracowanie	NOUN	SUBST	_	5	nsubj	_	_
2	redakcyjne	redakcyjny	ADJ	ADJ	_	1	amod	_	_
3	i	i	CCONJ	CONJ	_	4	cc	_	_
4	przypisy	przypis	NOUN	SUBST	_	2	conj	_	SpaceAfter=No
5	:	:	PUNCT	INTERP	_	0	ROOT	_	_
6	Paulina	paulin	NOUN	SUBST	_	5	nmod	_	_
7	Choromańska	choromańska	N